In [1]:
import pandas as pd
import fuzzywuzzy
from fuzzywuzzy import process
import os
import time

D:\Applications\Anaconda\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
pilot_list_df=pd.read_csv('pilot_info.csv',header=0)

In [3]:
pilot_list_df

,Unnamed: 0,code,url
0,0,NL,http://www.rodekruis.nl
1,1,MI,http://www.redcross.mw
2,2,GT,http://www.cruzroja.gt
3,3,LE,http://www.redcross.org.lb


In [4]:
corpus_mapping_df=pd.read_csv("pilot_scarping/corpus/mapping",header=None)

In [5]:
corpus_mapping_df[0].map(lambda x: [x[:x.find("http")],x[x.find("http"):]]).to_list()

[['0', 'https://www.rodekruis.nl/'],
 ['1', 'https://www.cruzroja.gt/'],
 ['2', 'https://www.redcross.mw/'],
 ['3', 'https://www.rodekruis.nl/hulp-in-nederland/'],
 ['4', 'https://www.rodekruis.nl/doe-mee/'],
 ['5', 'https://www.rodekruis.nl/hulp-wereldwijd/'],
 ['6', 'https://www.rodekruis.nl/over-ons/'],
 ['7', 'https://www.rodekruis.nl/doe-mee/doneren/'],
 ['8', 'https://www.rodekruis.nl/doe-mee/werken-bij-het-rode-kruis/'],
 ['9', 'https://www.rodekruis.nl/doe-mee/vrijwilligerswerk/'],
 ['10', 'https://www.rodekruis.nl/ehbo/ehbo-cursus/'],
 ['11', 'https://www.rodekruis.nl/ehbo/gratis-ehbo-app/'],
 ['12', 'https://www.rodekruis.nl/contact/'],
 ['13', 'https://www.rodekruis.nl/ehbo/'],
 ['14', 'https://www.rodekruis.nl/campagnes/moeders/'],
 ['15', 'https://www.rodekruis.nl/dossiers/venezuela/'],
 ['16',
  'https://www.rodekruis.nl/ehbo-wat-te-doen-bij/contact-met-eikenprocessierups/'],
 ['17',
  'https://www.rodekruis.nl/nieuwsbericht/vijf-vijanden-van-de-vierdaagseloper/'],
 ['18'

In [6]:
corpus_mapping_df=pd.DataFrame(corpus_mapping_df[0].map(lambda x: [x[:x.find("http")],x[x.find("http"):]]).to_list())

In [7]:
corpus_mapping_df.columns=['id','url']

In [63]:
def read_zipfile(dir_name,file_name):
    df_list=[]
    county_code=file_name
    path=dir_name+file_name
    for file in [file for file in os.listdir(path) if 'administrative' in file or "populatedplaces" in file or len(file)<7]:
        df_list.append(pd.read_csv(path+"/"+file,delimiter='\\t'))
    concected_df=pd.concat(df_list,axis=0)
    concected_df['code']=county_code
    concected_df.drop(columns=[i for i in concected_df.columns if len(i)>20],inplace =True)
    #print(concected_df)
    concected_df.fillna('',inplace=True)
    concected_df=concected_df[['ADM1', 'CC1', 'CC2', 'DMS_LAT', 'DMS_LONG', 'DSG', 'ELEV',
       'FC', 'FULL_NAME_ND_RG', 'FULL_NAME_ND_RO', 'FULL_NAME_RG',
       'FULL_NAME_RO','LC','SORT_NAME_RO']]
    #print(concected_df.columns)
    return concected_df['FULL_NAME_RO'].to_list()+concected_df['SORT_NAME_RO'].to_list()

In [97]:
location_dir="../locations/geonames_20190701_CntryFiles/"
def fuzzy_match_by_nation(inital_site,code,mapping_list):
    filtered_pages=mapping_list[mapping_list.apply(lambda x:True if inital_site in x.url else False,axis=1)]
    name_list=read_zipfile(location_dir,code.lower())
    count=0
    result_list=[]
    #print(filtered_pages)
    #print(name_list)
    for row in filtered_pages.iterrows():
        #print(row[1])
        start=time.time()
        row=row[1]
#         if count==1:
#             break
        with open("pilot_scarping/corpus/"+str(row['id']),'r',encoding='utf-8') as f:
            #print(f.read())
#             for line in f.readlines():
#                 result=process.extract(line,name_list,limit=10)
#                 print(result)
            text= f.read().lower()
            #print("Lilongwe".lower() in text)
            name_list=pd.Series(name_list)
            #print(name_list.apply)
            #print(name_list.apply(lambda x: x if x.lower() in text else None).dropna())
            matched_list=name_list.apply(lambda x: x if x.lower() in text else None).dropna()
            result_list.append((row['url'],matched_list.shape[0]))
        count+=1
        end=time.time()
        #print("It takes "+str(end-start))
    result_df=pd.DataFrame(result_list)
    print(result_df.sort_values(by=[1],ascending=False))
    return result_df.sort_values(by=[1],ascending=False)[0].to_list()[0]

In [98]:
result_list=[]
for row in pilot_list_df.iterrows():
    country=row[1]
    url=country['url']
    best_url=fuzzy_match_by_nation(url[url.find("www"):].replace("/",'/'),country['code'],corpus_mapping_df)
    result_list.append((country['code'],best_url))

D:\Applications\Anaconda\lib\site-packages\ipykernel_launcher.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


                                                     0     1
1          https://www.rodekruis.nl/hulp-in-nederland/  4858
23   https://www.rodekruis.nl/ehbo/ehbo-op-evenemen...  4501
133  https://www.rodekruis.nl/vacatures-vrijwillige...  2139
38   https://www.rodekruis.nl/hulp-in-nederland/ops...  1988
339  https://www.rodekruis.nl/persberichten-overzicht/  1520
86   https://www.rodekruis.nl/hulp-wereldwijd/voorb...  1520
35          https://www.rodekruis.nl/nieuws-overzicht/  1422
375  https://www.rodekruis.nl/formulier/periodiek-s...  1331
361  https://www.rodekruis.nl/nieuwsbericht/de-wate...  1325
15   https://www.rodekruis.nl/nieuwsbericht/vijf-vi...  1318
98    https://www.rodekruis.nl/over-ons/onze-historie/  1317
267  https://www.rodekruis.nl/doe-mee/voor-bedrijve...  1296
189  https://www.rodekruis.nl/nieuwsbericht/aan-de-...  1296
440       https://www.rodekruis.nl/vacatures/biketeam/  1279
439  https://www.rodekruis.nl/vacatures/logistieke-...  1267
93        https://www.ro

D:\Applications\Anaconda\lib\site-packages\ipykernel_launcher.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


                                                    0    1
84  https://www.redcross.mw/project/wash-program-p...  348
28                https://www.redcross.mw/what-we-do/  266
96  https://www.redcross.mw/causes_category/causes...  264
23  https://www.redcross.mw/cause/danish-red-cross...  258
88   https://www.redcross.mw/project/project-profile/  233
36                https://www.redcross.mw/our-causes/  228
0                            https://www.redcross.mw/  226
20  https://www.redcross.mw/cause/flood-response-o...  216
97  https://www.redcross.mw/causes_category/causes...  212
98  https://www.redcross.mw/causes_category/causes...  212
82            https://www.redcross.mw/project/icbhpp/  208
72         https://www.redcross.mw/category/lifeline/  204
77  https://www.redcross.mw/cause/malawi-red-cross...  200
74  https://www.redcross.mw/cause/government-appre...  192
19  https://www.redcross.mw/cause/kuwait-red-cresc...  186
81  https://www.redcross.mw/cause/malawi-red-cross...  1

D:\Applications\Anaconda\lib\site-packages\ipykernel_launcher.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


                                                     0    1
6                 https://www.cruzroja.gt/contactanos/  819
3                https://www.cruzroja.gt/delegaciones/  553
4                    https://www.cruzroja.gt/noticias/  485
69            https://www.cruzroja.gt/noticias/page/4/  468
79   https://www.cruzroja.gt/noticias/boletin-infor...  446
29   https://www.cruzroja.gt/noticias/ingeniera-pat...  433
84            https://www.cruzroja.gt/noticias/page/5/  399
59   https://www.cruzroja.gt/noticias/asistencia-hu...  382
27   https://www.cruzroja.gt/noticias/annabella-fol...  374
77   https://www.cruzroja.gt/noticias/boletin-infor...  360
49            https://www.cruzroja.gt/atencion-medica/  354
83   https://www.cruzroja.gt/noticias/boletin-infor...  332
82   https://www.cruzroja.gt/noticias/lo-que-el-vie...  317
32            https://www.cruzroja.gt/noticias/page/2/  312
57   https://www.cruzroja.gt/noticias/trece-dias-de...  303
34            https://www.cruzroja.gt/no

D:\Applications\Anaconda\lib\site-packages\ipykernel_launcher.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


                                                     0    1
335  http://www.redcross.org.lb/SubPage.aspx?pageid...  293
36   http://www.redcross.org.lb/NewsDetails.aspx?pa...  268
345  http://www.redcross.org.lb/NewsDetails.aspx?pa...  255
80   http://www.redcross.org.lb/NewsDetails.aspx?pa...  254
234  http://www.redcross.org.lb/NewsDetails.aspx?pa...  254
346  http://www.redcross.org.lb/SubPage.aspx?pageid...  252
154  http://www.redcross.org.lb/NewsDetails.aspx?pa...  252
78   http://www.redcross.org.lb/NewsDetails.aspx?pa...  248
192  http://www.redcross.org.lb/NewsDetails.aspx?pa...  246
132  http://www.redcross.org.lb/NewsDetails.aspx?pa...  246
7    http://www.redcross.org.lb/SubPage.aspx?pageid...  245
144  http://www.redcross.org.lb/NewsDetails.aspx?pa...  244
311  http://www.redcross.org.lb/NewsDetails.aspx?pa...  244
306  http://www.redcross.org.lb/NLRegister.aspx?pag...  243
133  http://www.redcross.org.lb/NewsDetails.aspx?pa...  243
40   http://www.redcross.org.lb/NewsDeta

In [92]:
fuzzy_match_by_nation("www.redcross.mw","MI",corpus_mapping_df)

D:\Applications\Anaconda\lib\site-packages\ipykernel_launcher.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


id                            2
url    https://www.redcross.mw/
Name: 2, dtype: object
It takes 0.10774087905883789
id                                           33
url    https://www.redcross.mw/where-we-work-2/
Name: 33, dtype: object
It takes 0.05784487724304199
id                                      34
url    https://www.redcross.mw/who-we-are/
Name: 34, dtype: object
It takes 0.09375119209289551
id                                  35
url    https://www.redcross.mw/health/
Name: 35, dtype: object
It takes 0.05684614181518555
id                                               36
url    https://www.redcross.mw/disaster-management/
Name: 36, dtype: object
It takes 0.059870243072509766
id                                     37
url    https://www.redcross.mw/vacancies/
Name: 37, dtype: object
It takes 0.2868974208831787
id                                      38
url    https://www.redcross.mw/contact-us/
Name: 38, dtype: object
It takes 0.06482410430908203
id                              

It takes 0.05385303497314453
id                                                   777
url    https://www.redcross.mw/galleries/countries-wi...
Name: 777, dtype: object
It takes 0.05385446548461914
id                                                  778
url    https://www.redcross.mw/gallery_category/people/
Name: 778, dtype: object
It takes 0.06534743309020996
id                                                   779
url    https://www.redcross.mw/galleries/countries-wi...
Name: 779, dtype: object
It takes 0.05884218215942383
id                                                   780
url    https://www.redcross.mw/gallery_category/workers/
Name: 780, dtype: object
It takes 0.05984044075012207
id                                      781
url    https://www.redcross.mw/team/michel/
Name: 781, dtype: object
It takes 0.08078479766845703
id                                     782
url    https://www.redcross.mw/team/jerry/
Name: 782, dtype: object
It takes 0.0718085765838623
id                  

In [93]:
fuzzy_match_by_nation("https://www.rodekruis.nl","NL",corpus_mapping_df)

D:\Applications\Anaconda\lib\site-packages\ipykernel_launcher.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


id                             0
url    https://www.rodekruis.nl/
Name: 0, dtype: object
It takes 1.080110788345337
id                                               3
url    https://www.rodekruis.nl/hulp-in-nederland/
Name: 3, dtype: object
It takes 2.5233240127563477
id                                     4
url    https://www.rodekruis.nl/doe-mee/
Name: 4, dtype: object
It takes 0.9235293865203857
id                                             5
url    https://www.rodekruis.nl/hulp-wereldwijd/
Name: 5, dtype: object
It takes 1.075162649154663
id                                      6
url    https://www.rodekruis.nl/over-ons/
Name: 6, dtype: object
It takes 1.1799123287200928
id                                             7
url    https://www.rodekruis.nl/doe-mee/doneren/
Name: 7, dtype: object
It takes 0.9156115055084229
id                                                     8
url    https://www.rodekruis.nl/doe-mee/werken-bij-he...
Name: 8, dtype: object
It takes 0.9195401668548584
i

It takes 0.9446549415588379
id                                                   466
url    https://www.rodekruis.nl/ehbo/ehbo-cursus/rean...
Name: 466, dtype: object
It takes 0.934138298034668
id                                                   467
url    https://www.rodekruis.nl/ehbo-wat-te-doen-bij/...
Name: 467, dtype: object
It takes 0.9090225696563721
id                                                   468
url    https://www.rodekruis.nl/ehbo/ehbo-cursus/ehbo...
Name: 468, dtype: object
It takes 0.957439661026001
id                                                   469
url    https://www.rodekruis.nl/ehbo-wat-te-doen-bij/...
Name: 469, dtype: object
It takes 0.8896217346191406
id                                                   470
url    https://www.rodekruis.nl/ehbo-wat-te-doen-bij/...
Name: 470, dtype: object
It takes 0.890617847442627
id                                                   471
url    https://www.rodekruis.nl/ehbo-wat-te-doen-bij/...
Name: 471, dtype: object
I

It takes 0.9090650081634521
id                                                   516
url    https://www.rodekruis.nl/persoonlijk-verhaal/t...
Name: 516, dtype: object
It takes 0.9067974090576172
id                                                   517
url    https://www.rodekruis.nl/persoonlijk-verhaal/s...
Name: 517, dtype: object
It takes 0.933504581451416
id                                                   518
url    https://www.rodekruis.nl/persoonlijk-verhaal/j...
Name: 518, dtype: object
It takes 0.9235305786132812
id                                                   519
url    https://www.rodekruis.nl/persoonlijk-verhaal/d...
Name: 519, dtype: object
It takes 0.9404852390289307
id                                                   520
url    https://www.rodekruis.nl/formulier/donatiebedr...
Name: 520, dtype: object
It takes 0.8786485195159912
id                                                   521
url    https://www.rodekruis.nl/formulier/wijzigen-va...
Name: 521, dtype: object

It takes 1.0322849750518799
id                                                   566
url    https://www.rodekruis.nl/ehbo-wat-te-doen-bij/...
Name: 566, dtype: object
It takes 0.9225325584411621
id                                                   567
url    https://www.rodekruis.nl/ehbo-wat-te-doen-bij/...
Name: 567, dtype: object
It takes 0.9450130462646484
id                                                   568
url    https://www.rodekruis.nl/ehbo-wat-te-doen-bij/...
Name: 568, dtype: object
It takes 1.0290040969848633
id                                                   569
url    https://www.rodekruis.nl/ehbo-wat-te-doen-bij/...
Name: 569, dtype: object
It takes 0.8921928405761719
id                                                   570
url    https://www.rodekruis.nl/ehbo-wat-te-doen-bij/...
Name: 570, dtype: object
It takes 0.8831503391265869
id                                                   571
url    https://www.rodekruis.nl/ehbo-wat-te-doen-bij/...
Name: 571, dtype: objec

...

It takes 0.8966348171234131
id                                      1628
url    https://www.rodekruis.nl/stad/veghel/
Name: 1628, dtype: object
It takes 0.8512375354766846
id                                                  1629
url    https://www.rodekruis.nl/wp-content/uploads/20...
Name: 1629, dtype: object
It takes 0.025930404663085938
id                                                  1630
url    https://www.rodekruis.nl/vacatures/noodhulptea...
Name: 1630, dtype: object
It takes 1.0372259616851807
id                                                  1631
url    https://www.rodekruis.nl/wp-content/uploads/20...
Name: 1631, dtype: object
It takes 0.02692699432373047
id                                                  1632
url    https://www.rodekruis.nl/wp-content/uploads/20...
Name: 1632, dtype: object
It takes 0.025931596755981445
id                                                  1633
url    https://www.rodekruis.nl/wp-content/uploads/20...
Name: 1633, dtype: object
It takes 0.

It takes 0.9869132041931152
id                                                  1678
url    https://www.rodekruis.nl/persbericht/driekwart...
Name: 1678, dtype: object
It takes 1.0284204483032227
id                                                  1679
url    https://www.rodekruis.nl/persbericht/tweede-on...
Name: 1679, dtype: object
It takes 0.985417366027832
id                                            1680
url    https://www.rodekruis.nl/author/svanhouten/
Name: 1680, dtype: object
It takes 0.854759931564331
id                                                  1681
url    https://www.rodekruis.nl/persbericht/rode-krui...
Name: 1681, dtype: object
It takes 0.9913301467895508
id                                                  1682
url    https://www.rodekruis.nl/persbericht/rode-krui...
Name: 1682, dtype: object
It takes 0.9534506797790527
id                                                  1683
url    https://www.rodekruis.nl/persbericht/ontvoerde...
Name: 1683, dtype: object
It tak

It takes 0.9938676357269287
id                                                  1728
url    https://www.rodekruis.nl/persbericht/rode-krui...
Name: 1728, dtype: object
It takes 1.056220531463623
id                                                  1729
url    https://www.rodekruis.nl/persbericht/directeur...
Name: 1729, dtype: object
It takes 1.0038275718688965
id                                                  1730
url    https://www.rodekruis.nl/persbericht/nederland...
Name: 1730, dtype: object
It takes 1.0132167339324951
id                                                  1731
url    https://www.rodekruis.nl/persbericht/mensen-ke...
Name: 1731, dtype: object
It takes 1.0135819911956787
id                                                  1732
url    https://www.rodekruis.nl/persbericht/3fm-djs-r...
Name: 1732, dtype: object
It takes 1.0142881870269775
id                                                  1733
url    https://www.rodekruis.nl/persbericht/hilali-ni...
Name: 1733, dtype: 

It takes 1.0481960773468018
id                                                  1777
url    https://www.rodekruis.nl/persbericht/3fm-serio...
Name: 1777, dtype: object
It takes 1.0551931858062744
id                                                  1778
url    https://www.rodekruis.nl/persbericht/rode-krui...
Name: 1778, dtype: object
It takes 0.9813754558563232
id                                                  1779
url    https://www.rodekruis.nl/persbericht/rode-krui...
Name: 1779, dtype: object
It takes 0.9723999500274658
id                                                  1780
url    https://www.rodekruis.nl/persbericht/rode-krui...
Name: 1780, dtype: object
It takes 1.0452239513397217
id                                                  1781
url    https://www.rodekruis.nl/persbericht/rode-krui...
Name: 1781, dtype: object
It takes 1.0193290710449219
id                                                  1782
url    https://www.rodekruis.nl/persbericht/rode-krui...
Name: 1782, dtype:

It takes 0.9412581920623779
id                                                  1899
url    https://www.rodekruis.nl/nieuwsbericht/stap-aa...
Name: 1899, dtype: object
It takes 1.0384912490844727
id                                                  1900
url    https://www.rodekruis.nl/nieuwsbericht/longrea...
Name: 1900, dtype: object
It takes 1.3185057640075684
id                                                  1901
url    https://www.rodekruis.nl/nieuwsbericht/hulp-na...
Name: 1901, dtype: object
It takes 1.1090333461761475
                                                     0     1
1          https://www.rodekruis.nl/hulp-in-nederland/  4858
23   https://www.rodekruis.nl/ehbo/ehbo-op-evenemen...  4501
133  https://www.rodekruis.nl/vacatures-vrijwillige...  2139
38   https://www.rodekruis.nl/hulp-in-nederland/ops...  1988
339  https://www.rodekruis.nl/persberichten-overzicht/  1520
86   https://www.rodekruis.nl/hulp-wereldwijd/voorb...  1520
35          https://www.rodekruis.nl/nieu

In [94]:
fuzzy_match_by_nation("www.cruzroja.gt","GT",corpus_mapping_df)

D:\Applications\Anaconda\lib\site-packages\ipykernel_launcher.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


id                            1
url    https://www.cruzroja.gt/
Name: 1, dtype: object
It takes 0.6762025356292725
id                                    60
url    https://www.cruzroja.gt/nosotros/
Name: 60, dtype: object
It takes 0.9165942668914795
id                                    61
url    https://www.cruzroja.gt/acciones/
Name: 61, dtype: object
It takes 0.6547856330871582
id                                        62
url    https://www.cruzroja.gt/delegaciones/
Name: 62, dtype: object
It takes 0.8996098041534424
id                                    63
url    https://www.cruzroja.gt/noticias/
Name: 63, dtype: object
It takes 0.7181096076965332
id                                     64
url    https://www.cruzroja.gt/donativos/
Name: 64, dtype: object
It takes 0.46774983406066895
id                                       65
url    https://www.cruzroja.gt/contactanos/
Name: 65, dtype: object
It takes 0.6947150230407715
id                                                    66
url    

It takes 0.6897132396697998
id                                                  858
url    https://www.cruzroja.gt/desarrollo-delegaciones/
Name: 858, dtype: object
It takes 0.6223554611206055
id                                                   859
url    https://www.cruzroja.gt/empleos/facilitador-co...
Name: 859, dtype: object
It takes 0.5345509052276611
id                                                   872
url    https://www.cruzroja.gt/acceso-a-la-informacio...
Name: 872, dtype: object
It takes 0.684009313583374
id                                                  1825
url    https://www.cruzroja.gt/noticias/trece-dias-de...
Name: 1825, dtype: object
It takes 0.6393678188323975
id                                                  1826
url    https://www.cruzroja.gt/noticias/asistencia-hu...
Name: 1826, dtype: object
It takes 0.6606650352478027
id                                                  1827
url    https://www.cruzroja.gt/noticias/asistencia-hu...
Name: 1827, dtype: objec

It takes 0.45827198028564453
id                                                  1877
url    https://www.cruzroja.gt/informes-financieros/i...
Name: 1877, dtype: object
It takes 0.45478177070617676
id                                                  1878
url    https://www.cruzroja.gt/informes-financieros/i...
Name: 1878, dtype: object
It takes 0.4543900489807129
id                                                  1879
url    https://www.cruzroja.gt/informes-financieros/i...
Name: 1879, dtype: object
It takes 0.43909788131713867
id                                                  1880
url    https://www.cruzroja.gt/informes-financieros/i...
Name: 1880, dtype: object
It takes 0.4239840507507324
id                                                  1881
url    https://www.cruzroja.gt/informes-financieros/i...
Name: 1881, dtype: object
It takes 0.443375825881958
id                                                  1882
url    https://www.cruzroja.gt/informes-financieros/i...
Name: 1882, dtyp

In [95]:
fuzzy_match_by_nation("www.redcross.org.lb","LE",corpus_mapping_df)

D:\Applications\Anaconda\lib\site-packages\ipykernel_launcher.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


id                                                   28
url    http://www.redcross.org.lb/index.aspx?pageid=907
Name: 28, dtype: object
It takes 1.819312334060669
id                                                    99
url    http://www.redcross.org.lb/indexar.aspx?pageid...
Name: 99, dtype: object
It takes 1.0453121662139893
id                                                   100
url    http://www.redcross.org.lb/SubPage.aspx?pageid...
Name: 100, dtype: object
It takes 2.5261995792388916
id                                                   101
url    http://www.redcross.org.lb/SubPage.aspx?pageid...
Name: 101, dtype: object
It takes 1.81032133102417
id                                                   102
url    http://www.redcross.org.lb/SubPage.aspx?pageid...
Name: 102, dtype: object
It takes 1.8179428577423096
id                                                   103
url    http://www.redcross.org.lb/SubPage.aspx?pageid...
Name: 103, dtype: object
It takes 2.0315680503845215
id   

It takes 1.9477951526641846
id                                                   148
url    http://www.redcross.org.lb/NewsDetails.aspx?pa...
Name: 148, dtype: object
It takes 1.8102412223815918
id                                                   149
url    http://www.redcross.org.lb/NewsDetails.aspx?pa...
Name: 149, dtype: object
It takes 1.872096300125122
id                                                   150
url    http://www.redcross.org.lb/Index.aspx?pageid=6...
Name: 150, dtype: object
It takes 1.8182907104492188
id                                                   151
url    http://www.redcross.org.lb/NewsDetails.aspx?pa...
Name: 151, dtype: object
It takes 1.8719944953918457
id                                                   152
url    http://www.redcross.org.lb/NewsDetails.aspx?pa...
Name: 152, dtype: object
It takes 1.9119198322296143
id                                                   153
url    http://www.redcross.org.lb/Index.aspx?pageid=6...
Name: 153, dtype: object

It takes 1.8063933849334717
id                                                   198
url    http://www.redcross.org.lb/NewsDetails.aspx?pa...
Name: 198, dtype: object
It takes 1.833238124847412
id                                                   199
url    http://www.redcross.org.lb/NewsDetails.aspx?pa...
Name: 199, dtype: object
It takes 1.74643874168396
id                                                   200
url    http://www.redcross.org.lb/NewsDetails.aspx?pa...
Name: 200, dtype: object
It takes 1.8370754718780518
id                                                   201
url    http://www.redcross.org.lb/NewsDetails.aspx?pa...
Name: 201, dtype: object
It takes 1.924854040145874
id                                                   202
url    http://www.redcross.org.lb/NewsDetails.aspx?pa...
Name: 202, dtype: object
It takes 1.9958839416503906
id                                                   203
url    http://www.redcross.org.lb/NewsDetails.aspx?pa...
Name: 203, dtype: object
It

It takes 2.140260696411133
id                                                   248
url    http://www.redcross.org.lb/NewsDetails.aspx?pa...
Name: 248, dtype: object
It takes 1.8560616970062256
id                                                   249
url    http://www.redcross.org.lb/NewsDetails.aspx?pa...
Name: 249, dtype: object
It takes 1.8341209888458252
id                                                   250
url    http://www.redcross.org.lb/NewsDetails.aspx?pa...
Name: 250, dtype: object
It takes 1.9208643436431885
id                                                   251
url    http://www.redcross.org.lb/NewsDetails.aspx?pa...
Name: 251, dtype: object
It takes 2.0944039821624756
id                                                   252
url    http://www.redcross.org.lb/NewsDetails.aspx?pa...
Name: 252, dtype: object
It takes 2.0545055866241455
id                                                   253
url    http://www.redcross.org.lb/NewsDetails.aspx?pa...
Name: 253, dtype: object

...

It takes 1.1618926525115967
id                                                  1024
url    http://www.redcross.org.lb/NewsDetailsAr.aspx?...
Name: 1024, dtype: object
It takes 1.1229956150054932
id                                                  1025
url    http://www.redcross.org.lb/NewsDetailsAr.aspx?...
Name: 1025, dtype: object
It takes 1.1968104839324951
id                                                  1026
url    http://www.redcross.org.lb/NewsDetailsAr.aspx?...
Name: 1026, dtype: object
It takes 1.1469447612762451
id                                                  1027
url    http://www.redcross.org.lb/NewsDetailsAr.aspx?...
Name: 1027, dtype: object
It takes 1.162858009338379
id                                                  1028
url    http://www.redcross.org.lb/NewsDetailsAr.aspx?...
Name: 1028, dtype: object
It takes 1.1349644660949707
id                                                  1029
url    http://www.redcross.org.lb/NewsDetailsAr.aspx?...
Name: 1029, dtype: 

It takes 1.17539381980896
id                                                  1073
url    http://www.redcross.org.lb/NewsDetailsAr.aspx?...
Name: 1073, dtype: object
It takes 1.2316575050354004
id                                                  1074
url    http://www.redcross.org.lb/NewsDetailsAr.aspx?...
Name: 1074, dtype: object
It takes 1.0591683387756348
id                                                  1075
url    http://www.redcross.org.lb/NewsDetailsAr.aspx?...
Name: 1075, dtype: object
It takes 1.1274943351745605
id                                                  1076
url    http://www.redcross.org.lb/NewsDetailsAr.aspx?...
Name: 1076, dtype: object
It takes 1.1539218425750732
id                                                  1077
url    http://www.redcross.org.lb/NewsDetailsAr.aspx?...
Name: 1077, dtype: object
It takes 1.1230723857879639
id                                                  1078
url    http://www.redcross.org.lb/NewsDetailsAr.aspx?...
Name: 1078, dtype: o

It takes 1.2242426872253418
id                                                  1122
url    http://www.redcross.org.lb/NewsDetailsAr.aspx?...
Name: 1122, dtype: object
It takes 1.4381823539733887
id                                                  1123
url    http://www.redcross.org.lb/NewsDetailsAr.aspx?...
Name: 1123, dtype: object
It takes 1.3385248184204102
id                                                  1124
url    http://www.redcross.org.lb/NewsDetailsAr.aspx?...
Name: 1124, dtype: object
It takes 1.270432472229004
id                                                  1125
url    http://www.redcross.org.lb/NewsDetailsAr.aspx?...
Name: 1125, dtype: object
It takes 1.2830793857574463
id                                                  1126
url    http://www.redcross.org.lb/NewsDetailsAr.aspx?...
Name: 1126, dtype: object
It takes 1.2073862552642822
id                                                  1127
url    http://www.redcross.org.lb/NewsDetailsAr.aspx?...
Name: 1127, dtype: 

It takes 1.2885544300079346
id                                                  1171
url    http://www.redcross.org.lb/CategoryNews.aspx?p...
Name: 1171, dtype: object
It takes 1.8949334621429443
id                                                  1172
url    http://www.redcross.org.lb/SubPageAr.aspx?page...
Name: 1172, dtype: object
It takes 1.6639313697814941
id                                                  1173
url    http://www.redcross.org.lb/SubPageAr.aspx?page...
Name: 1173, dtype: object
It takes 1.2910592555999756
id                                                  1174
url    http://www.redcross.org.lb/SubPageAr.aspx?page...
Name: 1174, dtype: object
It takes 1.357377529144287
id                                                  1175
url    http://www.redcross.org.lb/SubPageAr.aspx?page...
Name: 1175, dtype: object
It takes 1.375835657119751
id                                                  1176
url    http://www.redcross.org.lb/SubPageAr.aspx?page...
Name: 1176, dtype: o

It takes 0.06633377075195312
id                                                  1926
url    http://www.redcross.org.lb/ContentPhotos/Photo...
Name: 1926, dtype: object
It takes 0.06682229042053223
id                                                  1927
url    http://www.redcross.org.lb/ContentPhotos/Photo...
Name: 1927, dtype: object
It takes 0.06482672691345215
id                                                  1928
url    http://www.redcross.org.lb/ContentPhotos/Photo...
Name: 1928, dtype: object
It takes 0.06482553482055664
id                                                  1929
url    http://www.redcross.org.lb/ContentPhotos/Photo...
Name: 1929, dtype: object
It takes 0.06879639625549316
id                                                  1930
url    http://www.redcross.org.lb/PhotoGallery.aspx?p...
Name: 1930, dtype: object
It takes 1.7493219375610352
id                                                  1931
url    http://www.redcross.org.lb/ContentPhotos/Photo...
Name: 1931, d

In [23]:

corpus_mapping_df[corpus_mapping_df.apply(lambda x:True if "www.redcross.mw" in x.url else False,axis=1)]
#"https://www.redcross.mw/" in corpus_mapping_df.url

,id,url
2,2,https://www.redcross.mw/
33,33,https://www.redcross.mw/where-we-work-2/
34,34,https://www.redcross.mw/who-we-are/
35,35,https://www.redcross.mw/health/
36,36,https://www.redcross.mw/disaster-management/
37,37,https://www.redcross.mw/vacancies/
38,38,https://www.redcross.mw/contact-us/
39,39,https://www.redcross.mw/about-us/
40,40,https://www.redcross.mw/basket/
41,41,https://www.redcross.mw/causes/


In [17]:
for i in corpus_mapping_df['url']:
    print("https://www.redcross.mw/" in i)

False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
True
True
True
True
True
True
True
True
True
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
F

In [81]:
test=pd.Series([1,2,3,5])

In [87]:
test.apply(lambda x: x if x in [1,5] else None).dropna().shape

(2,)

In [99]:
result_list

[('NL', 'https://www.rodekruis.nl/hulp-in-nederland/'),
 ('MI', 'https://www.redcross.mw/project/wash-program-profile/'),
 ('GT', 'https://www.cruzroja.gt/contactanos/'),
 ('LE', 'http://www.redcross.org.lb/SubPage.aspx?pageid=209&PID=162')]